In [1]:
import numpy as np  # useful for many scientific computing in Python
import pandas as pd # primary data structure library

<b> Installing Beautifulsoup4</b>

In [2]:
!conda install -c anaconda beautifulsoup4

Solving environment: done

# All requested packages already installed.



Need to get data from this address: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [3]:
from bs4 import BeautifulSoup

url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

import requests
response = requests.get(url)
#print (response.status_code) #Should be 200 for a good request
#print (response.content)

soup = BeautifulSoup(response.content, 'html.parser')


OK now we have soup! Let's pase what we got

In [4]:
gdp_table = soup.find("table", attrs={"class": "wikitable"})
gdp_table_data = gdp_table.tbody.find_all("tr") 
print("Number of rows = " + str(len(gdp_table_data)))

#let's get the headers
headings = []
for th in gdp_table_data[0].find_all("th"):
    # remove any newlines and extra spaces from left and right
    headings.append(th.string.replace('\n', ' ').strip())

print("The Table Headers are:")
print(headings)



Number of rows = 181
The Table Headers are:
['Postal code', 'Borough', 'Neighborhood']


In [5]:
df_q1 = pd.DataFrame(columns=headings)
contents = []

for row in gdp_table_data[1:]:
    myrow = []
    for td in row.find_all("td"):
        # remove any newlines and extra spaces from left and right
        myrow.append(td.string.replace('\n', ' ').strip())
    #If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
    if myrow[2] == "":
        myrow[2] = myrow[1]
    df_q1.loc[len(df_q1)] = myrow
    
df_q1["Borough"].replace("Not assigned", np.nan, inplace=True)
df_q1.dropna(subset=["Borough"], inplace=True)
df_q1.reset_index(drop=True, inplace=True)
print("Number of rows = ", str(len(df_q1)))
df_q1.tail()


Number of rows =  103


,Postal code,Borough,Neighborhood
98,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing CentrE
101,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...
102,M8Z,Etobicoke,Mimico NW / The Queensway West / South of Bloo...


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma


In [6]:
#Show the issue the question is talking about:
df_q1.loc[df_q1['Postal code'] == "M5A"]

,Postal code,Borough,Neighborhood
2,M5A,Downtown Toronto,Regent Park / Harbourfront


Hmmm, looks like they are already combined in the raw dataset. We can replace the "/" with a ",". Seems pointless, So here is code that would combine text from same rows with a ",", if they existed in seperate rows.

In [7]:
df_q1 = df_q1.groupby(['Postal code','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df_q1.tail()

,Postal code,Borough,Neighborhood
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,Kingsview Village / St. Phillips / Martin Grov...
101,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...
102,M9W,Etobicoke,Northwest


In [8]:
print("As you can see the number of rows is still " + str(len(df_q1)))

As you can see the number of rows is still 103


So we ansered the question of combining like zip code neighborhoods, but in this dataset it is unneccesary.

Finnaly, they want us to do this: 
In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [9]:
df_q1.shape

(103, 3)